# Aprendizaje Automático

## Trabajo final:Acoustic Extinguisher Fire

###### Igor Vons y Aitor Uranga

Importamos las librerias basicas que vamos a utilizar durante el trabajo

In [75]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from matplotlib.colors import ListedColormap
import time
from sklearn.metrics import confusion_matrix

### Lectura de datos

Primero, abrimos en documento de texto donde tenemos el dataset y lo procesamos para que podamos utilizarlo posteriormente sin problemas en los distintos modelos de clasificación. Después, cargamos los datos mediante la función load y asignamos las características y el resultado a las variables X e y respectivamente.

In [76]:
# Abrimos el archivo de texto original en modo lectura
with open('Acoustic_Extinguisher_Fire_Dataset.txt', 'r') as f:
  # Abrimos el nuevo archivo de texto en modo escritura
  with open('Acoustic_Extinguisher_Fire_Dataset_re.txt', 'w') as g:
    # Leemos el archivo original línea por línea
    for line in f:
        # Dividimos cada línea en campos
        fields = line.split(',')
        # Remplazamos la segunda columna de valores nominales a un digito segun su valor
        if fields[1] == 'gasoline':
            fields[1] = '1'
        elif fields[1] == 'thinner':
            fields[1] = '2'
        elif fields[1] == 'kerosene':
            fields[1] = '3'
        elif fields[1] == 'lpg':
            fields[1] = '4'
        # Escribimos la línea modificada en el nuevo archivo
        g.write(' '.join(fields))

In [77]:
data = np.loadtxt("Acoustic_Extinguisher_Fire_Dataset_re.txt", delimiter=' ')
X = data[:,:-1]
y = data[:,-1]

## Mostrar los datos

El dataset elegido cuenta con 6 características:

1. El tamaño de la llama que va de 1 a 7
2. El combustible usado, gasolina, diluyente, keroseno y lpg (Que anteriormente hemos pasado a una variable numérica
entre 1 y 4)
3. La distancia del altavoz a la llama de 10 a 190 cm
4. Los decibelios del altavoz de 72 a 113 dB
5. El flujo de aire de 0 a 17 m/s
6. La frecuencia del altavoz 1-75 Hz
    
Y un consecuente que es 0 o 1, 1 si la llama se ha apagado y 0 si no

Nota: Para poder visualizar los datos en 3d desde distintos angulos hemos usado %matplotlib qt5 que genera ventanas especiales. Es posible que el notebook pregunte por permisos de uso para su correcto funcionamiento

In [78]:
print(X[:4,:])
print(y[:4])

[[ 1.   1.  10.  96.   0.  75. ]
 [ 1.   1.  10.  96.   0.  72. ]
 [ 1.   1.  10.  96.   2.6 70. ]
 [ 1.   1.  10.  96.   3.2 68. ]]
[0. 1. 1. 1.]


In [79]:
# Habilitar gráficos interactivos en una ventana separada
%matplotlib qt5

# Crear los mapas de colores
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

# Crear la figura 3D
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

# Hacer el scatter plot usando la función Axes3D.scatter
ax.scatter(data[:, 0], data[:, 1], data[:, 2], c=y, cmap=cmap_bold)
ax.set_xlabel("Tamaño de llama", fontsize=12)
ax.set_ylabel("Tipo de combustible", fontsize=12)
ax.set_zlabel("distancia", fontsize=12)

# Mostrar el gráfico y mantener la ventana abierta
plt.show(block=True)


<IPython.core.display.Javascript object>

In [80]:
# Habilitar gráficos interactivos en una ventana separada
%matplotlib qt5

# Crear los mapas de colores
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

# Crear la figura 3D
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

# Hacer el scatter plot usando la función Axes3D.scatter
ax.scatter(data[:, 0], data[:, 1], data[:, 3], c=y, cmap=cmap_bold)
ax.set_xlabel("Tamaño de llama", fontsize=12)
ax.set_ylabel("Tipo de combustible", fontsize=12)
ax.set_zlabel("dB", fontsize=12)

# Mostrar el gráfico y mantener la ventana abierta
plt.show(block=True)

<IPython.core.display.Javascript object>

En las dos gráficas iniciales, se puede observar como el tamaño de la llama y el tipo de combustible no son tan importantes como la distancia y los dB. Se observa que (casi) independientemente de las primeras dos variables, el apagado o no de la llama viene principalmente determinado por el eje vertical de la gráfica.
- grafica1: Se puede observar que el cambio de clase sucede aproximadamente entre 100-75 en la variable distancia. Cabe remarcar que sí que parece que conforme más pequeña sea la llama, más probable es que se apague a mayor distancia, pero sigue teniendo mucho menos peso.
- grafica2: Este caso es más curioso porque aunque observamos un comportamiento parecido a la anterior, donde el eje vertical es el principal responsable del cambio, en lugar de haber un solo punto de cambio, hay 3. Parece ser que el fuego se apaga en torno a los 90-85 dB, pero entre 90 y 100 pierde eficacia, y por encima de los 100 la vuelve a recuperar. Nuevamente, un menor tamaño de llama parece indicar mayor flexibilidad sobre los valores de la intensidad de sonido que pueden apagar la llama (las franjas de apagados se vuelven más anchas).

In [81]:
# Habilitar gráficos interactivos en una ventana separada
%matplotlib qt5

# Crear los mapas de colores
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

# Crear la figura 3D
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

# Hacer el scatter plot usando la función Axes3D.scatter
ax.scatter(data[:, 0], data[:, 1], data[:, 4], c=y, cmap=cmap_bold)
ax.set_xlabel("Tamaño de llama", fontsize=12)
ax.set_ylabel("Tipo de combustible", fontsize=12)
ax.set_zlabel("Flujo de aire", fontsize=12)

# Mostrar el gráfico y mantener la ventana abierta
plt.show(block=True)

<IPython.core.display.Javascript object>

In [82]:
# Habilitar gráficos interactivos en una ventana separada
%matplotlib qt5

# Crear los mapas de colores
cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

# Crear la figura 3D
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')

# Hacer el scatter plot usando la función Axes3D.scatter
ax.scatter(data[:, 0], data[:, 1], data[:, 5], c=y, cmap=cmap_bold)
ax.set_xlabel("Tamaño de llama", fontsize=12)
ax.set_ylabel("Tipo de combustible", fontsize=12)
ax.set_zlabel("Frecuencia", fontsize=12)

# Mostrar el gráfico y mantener la ventana abierta
plt.show(block=True)

<IPython.core.display.Javascript object>

Similar a los casos anteriores, la variable vertical es la más relevantes en el caso Flujo de aire. En el de la Frecuencia, sin embargo, no lo es tan pronunciadamente como antes.
- grafica3: Vemos claramente que la frontera de cambio se encuentra claramente entre 5 y 7.5. También se puede observar que la influencia que tiene el tamaño de llama es bastante más acentuado del que hemos observado en las anteriores gráficas, aunque siguiendo si tener tanta relevancia como la tiene el flujo.
- grafica4: En comparación con las anteriores variables verticales, aquí frecuencia es bastante más inconsistente a la hora de determinar el apagado o no del fuego. El mejor desempeño lo encontramos entre 5 y 20 aproximadamente, aunque varía mucho. De hecho, casi se podría decir que tiene aproximadamente el mismo peso que el tamaño de la llama, dado que para llamas más pequeñas la efectividad se dispara hasta alcanzar rangos de 5-30Hz/5-40Hz.

Se puede concluir que en este dataset las variables más determinantes apuntan a ser los dB, la distancia y el flujo, quedando las otras 3 relegadas a un plano más secundario, aunque ni mucho menos nulo.
Cabe también destacar una anomalía que parece repetirse en todas las gráficas, ya es que la llama de tamaño 6 parece apagarse con mucha más facilidad que la 5, rompiendo la tendencia que uno esperaría ver. Esto se debe a que el tipo de combustible 4 es LPG, a diferencia de los anteriores que son combustibles líquidos, por lo que aquí no se mide el tamaño de la llama sino si la corriente de combustible está completamente abierta, o medio abierta. Es pòr eso que los datos apaercen separados del resto.

## Preprocesamiento de datos

Para empezar, antes de meternos con los distintos modelos de clasificación, cambiaremos la escala de cada característica de nuestro dataset para que esté entre 0 y 1, ya que el rango de valores que manejan las características son bastante distintas entre ellas, y luego partirlo en tres subconjuntos, train o entrenamiento, validación y test o prueba.

In [83]:
from sklearn.preprocessing import MinMaxScaler

#Ponemos una semilla para quitar la aleatoriedad entre pruebas
np.random.seed(123456789)

# Inicializamos el MinMaxScaler que sirve para normalizar los datos entre 0 y 1 y lo entrenamos con nuestro conjunto de datos
# Luego se lo aplicamos al conjunto de datos
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

m = len(X)

# Utiliza np.random.permutation para obtener la localización de los ejemplos aleatoriamente
# Luego aplica estos nuevos índices a X e y para reordenarlos aleatoriamente
t = np.random.permutation(np.arange(m))
X_scaled =X_scaled[t]
y = y[t]
# Establece el número de ejemplos en mtrain y mval como el 60 y 20% de ejemplos en X (utiliza el redondeo y recuerda
# que el resultado debe ser un entero)
mtrain = int(m*0.6)
mval = int(m*0.8)- int(m*0.6)

# Obtén el número de ejemplos en test (mtest) como el número de ejemplos restantes a particionar
mtest = m - (int(m*0.8))
# Crea los nuevos conjuntos
Xtrain = X_scaled[:mtrain]
Xval = X_scaled[mtrain:int(m*0.8)]
Xtest = X_scaled[int(m*0.8):m]
ytrain = y[:mtrain]
yval = y[mtrain:int(m*0.8)]
ytest = y[int(m*0.8):m]

## Modelos de clasificación

En este apartado vamos a analizar los distintos modelos de clasificación. Para esto, en cada modelo vamos a buscar los híper-parámetros que más se ajusten a nuestro dataset, entrenándolo con el conjunto de entrenamiento y utilizando el resultado en el conjunto de validación. Por último, comprobaremos el resultado en el conjunto de test.

### Regresión Logística

El primer modelo que vamos a analizar va a ser el de regresión logística, este se construye a partir de un conjunto de datos etiquetados, es decir, que ya sabemos a qué categoría pertenecen los objetos. El modelo se entrena para aprender a asociar ciertas características de los objetos con la categoría a la que pertenecen. Una vez entrenado, el modelo puede hacer predicciones sobre la categoría de objetos desconocidos.

A diferencia de la regresión lineal, que se utiliza para predecir valores continuos, la regresión logística se utiliza para predecir la probabilidad de que un evento ocurra.

Antes de aplicar la regresión logística, necesitamos añadir características polinomiales para conseguir una frontera no lineal.

Para esto, vamos a probar distintos valores de polinomios y distintos valores de lambda.

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures

np.random.seed(123456789)
# Inicializamos las variables en las cuales vamos a guardar los mejores valores del clasificador
bestacc = 0
bestlambda = 0
bestPoli = 0
# Inicilizamos los valores a observar de polinomios y lambdas
ValoresPoli = [3,4,6,8]
valoresLambda = np.array([0.1e-100,1,100])

for j,valorPoly in enumerate(ValoresPoli) :
    poly = PolynomialFeatures(valorPoly)
    XXval = poly.fit_transform(Xval)
    XXtrain = poly.fit_transform(Xtrain)
    XXtest = poly.fit_transform(Xtest)
    for i, valorLambda in enumerate(valoresLambda):
        # Se incializa el modelo llamando al constructor de la regresión lineal
        LogReg = LogisticRegression(penalty = 'l2', C = 1/valorLambda,max_iter = 400, solver = 'newton-cg')
        # Utiliza fit para ajustar el modelo a los datos X e y de entrenamiento
        LogReg.fit(XXtrain, ytrain)
        thetas = LogReg.coef_
        # Cálculo del porcentaje de acierto en el conjunto de validacion
        p = LogReg.predict(XXval) 
        aciertos = p==yval
        accuracy = (aciertos.sum()/p.size)*100
        if accuracy > bestacc:
            bestacc = accuracy
            bestlambda = valorLambda
            bestPoli = valorPoly

# Mostramos los valores de los mejores híper-parámetros  
print("Mejor valor de polinomio {} y mejor lambda {}".format(bestPoli,bestlambda))

C:\Users\ivons\miniconda3\envs\py38ml\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\ivons\miniconda3\envs\py38ml\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\ivons\miniconda3\envs\py38ml\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


Mejor valor de polinomio 8 y mejor lambda 1e-101


In [85]:
# Ahora con la mejor lambda y mejor valor de polinomio volvemos a inicializar y ajustar a los datos de entrenamiento
poly = PolynomialFeatures(bestPoli)
XXval = poly.fit_transform(Xval)
XXtrain = poly.fit_transform(Xtrain)
XXtest = poly.fit_transform(Xtest)
LogReg = LogisticRegression(penalty = 'l2', C = 1/bestlambda,max_iter = 400, solver = 'newton-cg')
LogReg.fit(XXtrain, ytrain)
# Cálculo del porcentaje de acierto para todos los casos
pred_train = LogReg.predict(XXtrain)
pred_val = LogReg.predict(XXval)
pred_test = LogReg.predict(XXtest)
accTrainLR = metrics.accuracy_score(pred_train, ytrain)
accTestLR = metrics.accuracy_score(pred_test, ytest)
accValLR = metrics.accuracy_score(pred_val, yval)
print("Precisión en train: {}".format(accTrainLR))
print("Precisión en test: {}".format(accTestLR))
print("Precisión en val: {}".format(accValLR))

Precisión en train: 0.9940754897276637
Precisión en test: 0.9627400401261106
Precisión en val: 0.9673165137614679


C:\Users\ivons\miniconda3\envs\py38ml\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


### Redes Neuronales

   El segundo, va a ser una red neuronal.  Estos modelos están formados por una o más capas de "neuronas", que son unidades de cálculo que reciben entradas, las combinan de alguna manera y producen una salida. 
   
   Como en este caso, el unico híper-parámetros es la cantidad de capas ocultas tenemos, y cuantas mas capas haya el porcentaje ira subiendo lentamente hasta estancarse, buscaremos donde el error disminuya poco y el aumento de capas conforme a la mejora no sea optimo. 

In [86]:
from sklearn.neural_network import MLPClassifier

np.random.seed(123456789)
# Crea el clasificador con size neuronas en la capa oculta.
# Después entrénalo, y obten la precisión en los datos de entrenamiento.
# Guarda la precisión final en una variable accTrain.
hidden_sizes = range(1, 50, 2)
accuracies = np.zeros(len(hidden_sizes))
bestacc = 1000
bestlayer = 1
for s, size in enumerate(hidden_sizes):
    # Crea el clasificador con size neuronas en la capa oculta.
    # Después entrénalo, y obtenemos el error en el conjunto de validacion.
    modelo = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = size, max_iter = 5000).fit(Xtrain,ytrain)
    y_pred = modelo.predict(Xval)
    accVal = metrics.mean_squared_error(yval,y_pred)
    accuracies[s] = accVal
    # Comparamos con el error minimo y comprobamos que sea menor y que haya una disminucion considerable.
    if (bestacc-accVal)>1e-3:
        # Si lo hay, guardamos el error menos y la cantidad de capas ocultas
        bestacc = accVal
        bestlayer = size

# Muestra en una gráfica como ha cambiado el error conforme aumentábamos hidden_sizes

plt.figure()
plt.plot(hidden_sizes,accuracies)
plt.xlabel("hidden layer sizes")
plt.ylabel("error")

# Mostramos la cantidad optima de capas ocultas
print("Cantidad optima de capas ocultas: {}".format(bestlayer))

<IPython.core.display.Javascript object>

Cantidad optima de capas ocultas: 35


In [87]:
# Por ultimo volvemos a inicializar el modelo con el híper-parámetros mas optimo
MLPC = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = bestlayer, max_iter = 5000)
MLPC.fit(Xtrain,ytrain)

# Predecimos los resultados para el modelo
pred_train = MLPC.predict(Xtrain)
pred_test = MLPC.predict(Xtest)
pred_val = MLPC.predict(Xval)

# Y calculamos la precision
accTrainNN = metrics.accuracy_score(pred_train, ytrain)
accTestNN = metrics.accuracy_score(pred_test, ytest)
accValNN = metrics.accuracy_score(pred_val, yval)
# Imprimimos los resultados obtenidos
print("Precisión en train: {}".format(accTrainNN))
print("Precisión en test: {}".format(accTestNN))
print("Precisión en val: {}".format(accValNN))

Precisión en train: 0.9546106067845198
Precisión en test: 0.9501289767841788
Precisión en val: 0.9481077981651376


### Naive bayes

El siguiente, va a ser Naive bayes, el cual es un algoritmo de aprendizaje automático que se utiliza para clasificar objetos en una de varias categorías basándose en características o cualidades de los objetos.

Este modelo se basa en el teorema de Bayes y el supuesto de independencia de las características.

En este caso no tiene ningun híper-parámetros y solo observaremos su precion para mas tarde comparar su rendimiento frente al resto de modelos.

In [88]:
from sklearn.naive_bayes import MultinomialNB
np.random.seed(123456789)
# Inicializamos el modelo
MultNB = MultinomialNB()

# Lo entrenamos
MultNB.fit(Xtrain, ytrain)
# Obtenemos las salidas para las instancias de train
pred_train = MultNB.predict(Xtrain)
# Obtenemos la precisión en train mediante metrics.accuracy_score (accTrain)
accTrainNB = metrics.accuracy_score(pred_train, ytrain)
# Obtenemos las salidas para val 
pred_val = MultNB.predict(Xval)
# Obtenemos la precisión en test mediante metrics.accuracy_score (accvañ)
accValNB = metrics.accuracy_score(pred_val, yval)
# Obtenemos las salidas para test 
pred_test = MultNB.predict(Xtest)
# Obtenemos la precisión en test mediante metrics.accuracy_score (accTest)
accTestNB = metrics.accuracy_score(pred_test, ytest)

# Imprimimos los resultados obtenidos
print("Precisión en train: {}".format(accTrainNB))
print("Precisión en test: {}".format(accTestNB))
print("Precisión en val: {}".format(accValNB))

Precisión en train: 0.8692785475394171
Precisión en test: 0.8779019776440241
Precisión en val: 0.8709862385321101


### Support Vector Machines

Luego tenemos el modelo Support Vector Machines funciona buscando una línea o un hiperplano que separe los objetos en distintas categorías de la manera más precisa posible.

La idea detrás de SVM es encontrar el hiperplano que maximiza el margen, es decir, la distancia entre los puntos más cercanos de cada categoría. Los puntos más cercanos a cada categoría se llaman vectores de soporte y son los que determinan el hiperplano óptimo.

En esta caos tenemos dos híper-parámetros a optimizar la C, si C es muy grande no regularizamos, y luego tenemos gamma, el cual afecta al kernel gausiano, que mide la similitud entre el ejemplo y su marcador, Menor gamma, la similitud tiende a 0 más rápidamente.

Para este caso, para calcular los híper-parámetros optimos tenemos 2 opciones. El primero, calcularlo nosotros manualmente viendo que híper-parámetros nos dan los mejores resultados para el conjunto de validacion.

In [89]:
from sklearn import svm
np.random.seed(123456789)

# Inicializamos los distintos valores de c y gamma que vamos a comprobar
C_values = [0.01, 0.03, 0.1, 0.3, 1, 3, 5, 10, 30, 100]
gamma_values = [0.01, 0.03, 0.1, 0.3, 1, 3, 5, 10, 30, 100]

# Inicializamos las variables donde guardaremos los híper-parámetros 
best_accVal = 0
best_params = {'C': None, 'gamma': None}

# Ponemos a contar el cronometro
start = time.perf_counter()

for C in C_values:
    for gamma in gamma_values:
        # Creamos la SVM con C=C y gamma=gamma
        svc = svm.SVC(kernel='rbf', C=C, gamma=gamma)
        # Entrenamos la SVM con los datos de train
        svc.fit(Xtrain, ytrain)
        # Obtenemos la precisión en los datos de validación
        accVal = svc.score(Xval, yval)
        
        # Si la precisión actual es mejor que la mejor encontrada hasta ahora...
        if accVal > best_accVal:
            # Guardamos la mejor precisión encontrada y los parámetros usados
            best_accVal = accVal
            best_params['C'] = C
            best_params['gamma'] = gamma

# Paramos el cronometro y vemos el tiempo trancurrido
end = time.perf_counter()
print('Tiempo trancurrido en segundos: ', end - start)        
# mostramos los parámetros seleccionados
print("Parámetros seleccionados: ", best_params)

Tiempo trancurrido en segundos:  349.34478970000055
Parámetros seleccionados:  {'C': 10, 'gamma': 30}


In [90]:
# Creamos una SVM con los mejores parámetros C y gamma encontrados
svc = svm.SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'])

# Crea y entrena un modelo SVC con C_best y gamma_best sobre los datos de train originales (X e y)
svc.fit(Xtrain, ytrain)

# Obtenemos la precisión en train y validación
accTrainSV = svc.score(Xtrain, ytrain)
accTestSV = svc.score(Xtest, ytest)
accValSV = svc.score(Xval, yval)
print("Precisión en train: {}".format(accTrainSV))
print("Precisión en test: {}".format(accTestSV))
print("Precisión en val: {}".format(accValSV))

Precisión en train: 0.9944577161968466
Precisión en test: 0.9704786471768415
Precisión en val: 0.9727637614678899


Segundo, mediante la funcion GridSearchCV de sklearn que nos busca sola los mejores híper-parámetros para el conjunto de datos dado.

In [91]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV


# Inicializamos los distintos valores de c y gamma que vamos a comprobar
C_values = [0.01, 0.03, 0.1, 0.3, 1, 3, 5, 10, 30, 100]
gamma_values = [0.01, 0.03, 0.1, 0.3, 1, 3, 5, 10, 30, 100]

# Genera el diccionario param_grid con los valores indicados
param_grid = {'C':C_values, 'gamma':gamma_values}

# Ponemos a contar al conometro
start = time.perf_counter()

# Genera el objeto ShuffleSplit para generar un único particionado fijo, con 20% de datos de validación y 
# random_state = 0
ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=0)

# Haz la llamada a la función GridSearchCV con un clasificador SVC y el diccionario de parámetros definido
# Para acelerar algo el entrenamiento, emplea también el parámetro n_jobs=-1, que hará que se aproveche mejor el
# procesador de tu equipo.
clasificadores = GridSearchCV(svm.SVC(), param_grid, n_jobs=-1, cv=ss)

# Entrena el modelo interno de GridSearchCV, y después obten los mejores valores de C y gamma en C_best y gamma_best
# Como la funcion nos hace la particion de datos de train y validacion ella misma,
# juntamos train y val antes de aplicar la funcion 
clasificadores.fit(np.vstack((Xtrain,Xval)), np.hstack((ytrain,yval)))
C_best, gamma_best = clasificadores.best_params_['C'], clasificadores.best_params_['gamma']
# Paramos el cronometro y vemos el tiempo trancurrido
end = time.perf_counter()
print('Tiempo trancurrido en segundos: ', end - start)
# Mostramos los mejores parametros
print("Mejores parámetros C: {}, gamma: {}".format(C_best, gamma_best))

Tiempo trancurrido en segundos:  143.3950390999962
Mejores parámetros C: 10, gamma: 30


In [92]:
# Crea y entrena un modelo SVC con C_best y gamma_best sobre los datos de train originales (X e y)
svc = svm.SVC(C=C_best, gamma=gamma_best)
svc.fit(Xtrain, ytrain)


# Obtenemos la precisión en train, test y validación
accTrainSV = svc.score(Xtrain, ytrain)
accTestSV = svc.score(Xtest, ytest)
accValSV = svc.score(Xval, yval)
print("Precisión en train: {}".format(accTrainSV))
print("Precisión en test: {}".format(accTestSV))
print("Precisión en val: {}".format(accValSV))

Precisión en train: 0.9944577161968466
Precisión en test: 0.9704786471768415
Precisión en val: 0.9727637614678899


Como podemos apreciar nos dan el mismo resultado pero la funcion dada por la libreria sklearn es bastante mas rapida.

### Ensembles OVO y OVA

Los ensembles combinan el resultado de varios clasificadores para obtener una predicción más precisa. La idea detrás de los clasificadores ensembles es que la combinación de varios clasificadores diferentes puede llevar a una mayor precisión que el uso de un solo clasificador.

Para estos clasificadores, primero vamos a comparar los ensembles y ver cuál es el mejor para nuestro dataset, comparando DecisionTree, Bagging, Boosting y Random Forest. Luego, veremos los modelos one-versus-one y one-versus-all.

In [93]:
from sklearn.tree import DecisionTreeClassifier
# Se importa la clase para realizar el ensemble con Bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

clasificadores = [DecisionTreeClassifier(criterion='entropy', random_state=12), 
                  BaggingClassifier(DecisionTreeClassifier(criterion='entropy'),n_estimators= 15, random_state=12)
                  ,AdaBoostClassifier(base_estimator = DecisionTreeClassifier(criterion='entropy'),n_estimators= 15, random_state=12),
                 RandomForestClassifier(criterion='entropy', max_features = "log2",n_estimators= 15, random_state=12)]
nombres = ['DecisionTree', 'Bagging', 'Boosting', 'RandomForest']
# Inicializar precisiones
bestclas = 0
accbestval=0
accbesttest=0
for i, clasificador in enumerate(clasificadores):
    # Construir clasificador, entrenarlo, obtener precisiones
    clasificador.fit(Xtrain,ytrain)
    accVal = clasificador.score(Xval,yval)*100
    # ¿Es el mejor?
    if (accVal>accbestval):
        accbestval = accVal
        bestclas = i
# Mostramos el mejor clasificador        
print("El mejor clasidicador emsembler es: {}".format(nombres[bestclas]))

El mejor clasidicador emsembler es: Bagging


In [99]:
#Entrenamos con el modelo definido y mostramos el accuracy
clasificadores[bestclas].fit(Xtrain,ytrain)
accValEN = clasificadores[bestclas].score(Xval,yval)
accTestEN = clasificadores[bestclas].score(Xtest,ytest)
accTrainEN = clasificadores[bestclas].score(Xtrain,ytrain)
print("Precisión en train: {}".format(accTrainEN))
print("Precisión en test: {}".format(accTestEN))
print("Precisión en val: {}".format(accValEN))

Precisión en train: 0.9987577639751553
Precisión en test: 0.9635998853539696
Precisión en val: 0.9664564220183486


En los ensemblers, tenemos como resultado que Bagging es el modelo que mas se ajusta a nuestro dataset.

Ahora, pasaremos a ver los modelo OVO y OVA

In [95]:
# Se importan las clase para OVA y para SVM
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# Se llama al constructor de OVA utilizando SVM como clasificador base
OVA = OneVsRestClassifier(SVC())
# Se entrena el modelo OVA
OVA.fit(Xtrain,ytrain)
# Se calcula el porcentaje de acierto de OVA
accTrainVA = OVA.score(Xtrain, ytrain)
accTestVA = OVA.score(Xtest, ytest)
accValVA = OVA.score(Xval, yval)
print("Precisión en train: {}".format(accTrainVA))
print("Precisión en test: {}".format(accTestVA))
print("Precisión en val: {}".format(accValVA))

Precisión en train: 0.9463927376970855
Precisión en test: 0.9461163657208369
Precisión en val: 0.9478211009174312


In [96]:
# Se importan las clase para OVO
from sklearn.multiclass import OneVsOneClassifier
# Se llama al constructor de OVO utilizando SVM como clasificador base
OVO = OneVsOneClassifier(SVC())
# Se entrena el modelo OVO
OVO.fit(Xtrain,ytrain)
# Se calcula el porcentaje de acierto de OVO
accTrainVO = OVO.score(Xtrain, ytrain)
accTestVO = OVO.score(Xtest, ytest)
accValVO = OVO.score(Xval, yval)
print("Precisión en train: {}".format(accTrainVO))
print("Precisión en test: {}".format(accTestVO))
print("Precisión en val: {}".format(accValVO))

Precisión en train: 0.9463927376970855
Precisión en test: 0.9461163657208369
Precisión en val: 0.9478211009174312


Aquí se puede apreciar como al tener solo dos resultados, apagado o encendido, aplicar one-versus-one sería lo mismo que aplicar one-versus-all, ya que como "all" son las clases restantes y solo hay una restante, la comparación acaba siendo la misma.


### Kmeans

En cuanto a Kmeans, aplicarlo no tendría mucho sentido, ya que buscaría los clústeres para dividir los datos, pero sabiendo cuantas clases resultantes tenemos y sabiendo a qué clases pertenecen, aplicar un modelo de aprendizaje no supervisado no tiene mucho sentido. El resultado que obtendríamos sería menos óptimo que modelos de clasificación supervisada.

## Resultado

Viendo los distintos modelos hay que mencionar que la mayoría de modelos tienen un porcentaje de acierto bastante elevado, con solo Naive bayes bajando del 90% de acierto y el resto no bajando del 94%. Esto nos muestra que todos los modelos, exceptuando quizás Naive bayes, se ajustan bastante bien a nuestro dataset y podrían ser utilizados.

Pero, el que mejores resultados nos ha dado es SVM ( Support Vector Machines), ya que nos da el mayor porcentaje de acierto en test después de medir los distintos hiper-parámetros. Siendo el modelo de clasificación que mejor se ajusta a nuestro problema, con una precisión en train de 99%, precisión en validación de 97,3% y precisión en test de 97%.

Puede haber varias razones para que nos dé como mejor clasificador el SVM estas son algunas de ellas:

SVM es un algoritmo de clasificación muy potente y es capaz de manejar conjuntos de datos muy grandes de manera eficiente.

SVM busca el hiperplano que maximiza el margen, es decir, la distancia entre los puntos más cercanos de cada categoría. Esto puede llevar a una mayor precisión en la clasificación.

SVM es un algoritmo que se adapta bien a datos que tienen una alta dimensionalidad (es decir, muchas características o variables). En este caso, SVM puede ser capaz de encontrar patrones complejos en los datos que otros algoritmos no pueden detectar.

En resumen, SVM es un algoritmo muy potente y es capaz de manejar conjuntos de datos muy grandes de manera eficiente, lo que lo hace ideal para problemas de clasificación binaria con muchos datos.

In [97]:
# Crea y entrena un modelo SVC con C_best y gamma_best sobre los datos de train originales (X e y)
from sklearn import svm
svc = svm.SVC(C=C_best, gamma=gamma_best)
svc.fit(Xtrain, ytrain)

# Obtenemos la precisión en train, test y validación
accTrainSV = svc.score(Xtrain, ytrain)
accTestSV = svc.score(Xtest, ytest)
accValSV = svc.score(Xval, yval)
print("Matriz de confusión:")
print(confusion_matrix(ytest, svc.predict(Xtest)))
print("Precisión en train con el modelo de clasificacion de SVM: {}".format(accTrainSV))
print("Precisión en test con el modelo de clasificacion de SVM: {}".format(accTestSV))
print("Precisión en val con el modelo de clasificacion de SVM: {}".format(accValSV))

Matriz de confusión:
[[1683   60]
 [  43 1703]]
Precisión en train con el modelo de clasificacion de SVM: 0.9944577161968466
Precisión en test con el modelo de clasificacion de SVM: 0.9704786471768415
Precisión en val con el modelo de clasificacion de SVM: 0.9727637614678899


In [132]:
# Data for the bar graph
model_names = ['Naive Bayes', 'SVM', 'OVA', 'OVO', 'Ensembles', 'Neural Network', 'Log Regression']
train_acc = [accTrainNB, accTrainSV, accTrainVA, accTrainVO, accTrainEN, accTrainNN, accTrainLR]
val_acc = [accValNB, accValSV, accValVA, accValVO, accValEN, accValNN, accValLR]
test_acc = [accTestNB, accTestSV, accTestVA, accTestVO, accTestEN, accTestNN, accTestLR]

# Set the width of each bar
bar_width = 0.2

# Transpose the data
model_names = np.array(model_names)
train_acc = np.array(train_acc)
val_acc = np.array(val_acc)
test_acc = np.array(test_acc)

# Set the position of each bar
bar_train = np.arange(len(train_acc))
bar_val = [x + bar_width for x in bar_train]
bar_test = [x + bar_width for x in bar_val]

# Create the figure and the axis
fig, ax = plt.subplots(figsize=(10, 5))

# Create the bars
plt.barh(bar_train, train_acc, height=bar_width, label='Train')
plt.barh(bar_val, val_acc, height=bar_width, label='Validation')
plt.barh(bar_test, test_acc, height=bar_width, label='Test')

for i, v in enumerate(train_acc):
    ax.annotate(str(round(v*100,2))+'%', xy=(v, i - .40), xytext=(5, 10), textcoords='offset points', ha='center', va='bottom')
for i, v in enumerate(val_acc):
    ax.annotate(str(round(v*100,2))+'%', xy=(v, i - .20), xytext=(5, 10), textcoords='offset points', ha='center', va='bottom')
for i, v in enumerate(test_acc):
    ax.annotate(str(round(v*100,2))+'%', xy=(v, i - .0), xytext=(5, 10), textcoords='offset points', ha='center', va='bottom')


# Add the labels for each bar
plt.yticks(np.arange(len(model_names)), model_names)

# Add the legend and the axis labels
plt.legend(fontsize=8)
plt.ylabel('Models')
plt.xlim((0, 1.2))
plt.subplots_adjust(left=0.4)
plt.xlabel('Accuracy')

# Show the plot
plt.show()


<IPython.core.display.Javascript object>